<a href="https://colab.research.google.com/github/MusadiqUrRahman/Quater_2/blob/main/Project_02_LangChain_RAG_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain_community pypdf -q langchain_google_genai  langchain_chroma

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 34.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.7/298.7 kB 24.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 38.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 71.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 22.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 72.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 10.1 MB/s eta 0:00

#**1: Loading Pdf**




In [2]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("/content/NIPS-2017-attention-is-all-you-need-Paper.pdf")
data = loader.load()


In [3]:
len(data)

11

# **2: Spliting text**

In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000)
docs = text_splitter.split_documents(data)


print("total number of documents : " ,len(docs))

total number of documents :  43


# **3 : Using API key**

In [5]:
from google.colab import userdata
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY_1')

# **4 : Creating embedding model**

In [6]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embedding = GoogleGenerativeAIEmbeddings(model="models/embedding-001" , google_api_key=GOOGLE_API_KEY)

# **5 : Embedding data and Retriver**

In [7]:
from langchain.vectorstores import Chroma
vectorstore = Chroma.from_documents(documents=docs, embedding=embedding)
my_retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 5})

# **6 : Creating LLM**

In [8]:
from langchain_google_genai import ChatGoogleGenerativeAI

my_llm = ChatGoogleGenerativeAI(model = "gemini-2.0-flash-exp" ,
                                google_api_key=GOOGLE_API_KEY,
                                temperature = 0.3 ,
                                max_tokens=1000,)
                                #convert_system_message_to_human=True

# **7 : Defining user_prompt**

In [9]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI



system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)


# **8 : Creating chains**

In [10]:
from langchain.chains import RetrievalQA
my_chain = RetrievalQA.from_chain_type(llm=my_llm,
                                       #chain_type="stuff",
                                       retriever=my_retriever,
                                       return_source_documents=True)

In [18]:
#question_answer_chain = create_stuff_documents_chain(my_llm, prompt)
#rag_chain = create_retrieval_chain(my_retriever, question_answer_chain)

# **9 : Asking question which is not in the pdf**

In [21]:
from IPython.display import Markdown

question = "who is imran khan"
answer = my_chain.invoke({"query": question})
Markdown(answer["result"])

This document does not contain any information about Imran Khan.

# **10 : Asking question which is in the pdf**

In [22]:
response = my_chain.invoke({"query" : "what is multi attention"})
Markdown(response["result"])

Multi-head attention allows the model to jointly attend to information from different representation subspaces at different positions. It consists of several attention layers running in parallel. The queries, keys, and values are linearly projected h times with different, learned linear projections to dk, dk, and dv dimensions, respectively. On each of these projected versions of queries, keys, and values, the attention function is performed in parallel, yielding dv-dimensional output values. These are concatenated and once again projected, resulting in the final values.